#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [2]:
import pandas as pd
import pymongo

In [3]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [4]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

c:\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [5]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [6]:
df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors='coerce')
df_ks = df_ks.dropna(subset=['launched'])
df_ks.rename(columns={'ID':'_id'}, inplace=True)

l_d =df_ks.to_dict(orient='records')

### Importer les données

In [8]:
from pymongo import MongoClient

client = MongoClient("localhost:27017")
db_ks = client.exercices
collection_kickstarter = db_ks['kickstarter'] 

collection_kickstarter.insert_many(l_d)

## Question 1  

In [17]:
cur = collection_kickstarter.find().sort("usd pledged",-1).limit(3)
list(cur)

[{'_id': 1701277493,
  'name': 'THE SHELLS ï¿½ AUSFLUG NACH NEU-FRIEDENWALD',
  'category': 'Immersive',
  'main_category': 'Theater',
  'currency': 'GBP',
  'deadline': '2015-04-19',
  'goal': '6500.0',
  'launched': datetime.datetime(2015, 3, 20, 19, 25, 56),
  'pledged': '6700.0',
  'state': 'successful',
  'backers': '89',
  'country': 'GB',
  'usd pledged': '9999.268739',
  'usd_pledged_real': 10130.027214998487},
 {'_id': 218218259,
  'name': 'True North ï¿½ Repurposed Sailcloth Products',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-14',
  'goal': '7500.0',
  'launched': datetime.datetime(2016, 5, 10, 15, 53, 56),
  'pledged': '9996.0',
  'state': 'successful',
  'backers': '50',
  'country': 'US',
  'usd pledged': '9996.0',
  'usd_pledged_real': 9996.0},
 {'_id': 1538904209,
  'name': 'Holo ï¿½ Smartphone & Tablet Hologram',
  'category': 'Gadgets',
  'main_category': 'Technology',
  'currency': 'AUD',
  'deadline': '2

## Question 2

In [16]:
cur = collection_kickstarter.find({"state" : "successful"})
list(cur)

[{'_id': 1688410639,
  'name': 'Sluff Off & Harald: Two latest EGGs are Classics "old & new"',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2016-07-19',
  'goal': 2000.0,
  'launched': datetime.datetime(2016, 7, 1, 21, 55, 54),
  'pledged': 7534.0,
  'state': 'successful',
  'backers': 254,
  'country': 'US',
  'usd pledged': 3796.0,
  'usd_pledged_real': 7534.0},
 {'_id': 156812982,
  'name': 'SketchPlanner: Create and Plan- all in one beautiful book!',
  'category': 'Art Books',
  'main_category': 'Publishing',
  'currency': 'USD',
  'deadline': '2017-09-27',
  'goal': 13000.0,
  'launched': datetime.datetime(2017, 8, 28, 15, 47, 2),
  'pledged': 16298.0,
  'state': 'successful',
  'backers': 367,
  'country': 'US',
  'usd pledged': 2670.0,
  'usd_pledged_real': 16298.0},
 {'_id': 1669374237,
  'name': 'Help I Am Wolves fund their first album "ABCD" on 12" vinyl!',
  'category': 'Music',
  'main_category': 'Music',
  'currency': 'EUR

## Question 3

In [28]:
cur = collection_kickstarter.aggregate([{ "$group" : { "_id": "$category", "count": { "$sum" : 1 } } } ])
list(cur)

[{'_id': 'Photo', 'count': 58},
 {'_id': 'Science Fiction', 'count': 274},
 {'_id': 'Painting', 'count': 1288},
 {'_id': 'Camera Equipment', 'count': 165},
 {'_id': 'Knitting', 'count': 78},
 {'_id': 'Puzzles', 'count': 95},
 {'_id': "Children's Books", 'count': 2686},
 {'_id': 'Photography', 'count': 2239},
 {'_id': 'Vegan', 'count': 229},
 {'_id': 'Chiptune', 'count': 10},
 {'_id': 'Film & Video', 'count': 4003},
 {'_id': 'Residencies', 'count': 32},
 {'_id': 'Comedy', 'count': 923},
 {'_id': 'Rock', 'count': 2707},
 {'_id': 'Stationery', 'count': 90},
 {'_id': 'Nonfiction', 'count': 3390},
 {'_id': 'Metal', 'count': 274},
 {'_id': 'Typography', 'count': 44},
 {'_id': 'Sculpture', 'count': 737},
 {'_id': 'Childrenswear', 'count': 192},
 {'_id': 'Zines', 'count': 144},
 {'_id': 'Playing Cards', 'count': 963},
 {'_id': 'Crafts', 'count': 1834},
 {'_id': 'Cookbooks', 'count': 217},
 {'_id': 'Makerspaces', 'count': 91},
 {'_id': 'Woodworking', 'count': 433},
 {'_id': 'Fabrication Tools',

## Question 4

In [143]:
import datetime
cur = collection_kickstarter.aggregate([
    {
      "$match": {
        "country": "FR",
        "launched": {"$lt": datetime.datetime.strptime("2016-01-01",'%Y-%m-%d')}
      }
    },
    {
      "$count" : "count"
    }
])
print(str(list(cur)[0]["count"]) + " projets lancés avant 2016 en France")

330 projets lancés avant 2016 en France


## Question 5

In [144]:
cur = collection_kickstarter.find({"country" : "US", "goal":{"$gt" : 200000}})
list(cur)

[{'_id': 866634482,
  'name': 'A CALL TO ADVENTURE',
  'category': 'Film & Video',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': '2012-09-14',
  'goal': 287000.0,
  'launched': datetime.datetime(2012, 8, 13, 23, 14, 2),
  'pledged': 1465.0,
  'state': 'failed',
  'backers': 11,
  'country': 'US',
  'usd pledged': 1465.0,
  'usd_pledged_real': 1465.0},
 {'_id': 993194166,
  'name': 'Storybricks, the storytelling online RPG',
  'category': 'Video Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2012-06-01',
  'goal': 250000.0,
  'launched': datetime.datetime(2012, 5, 1, 20, 49, 58),
  'pledged': 23680.54,
  'state': 'failed',
  'backers': 409,
  'country': 'US',
  'usd pledged': 23680.54,
  'usd_pledged_real': 23680.54},
 {'_id': 1147175344,
  'name': 'Shine On New World',
  'category': 'Theater',
  'main_category': 'Theater',
  'currency': 'USD',
  'deadline': '2013-11-04',
  'goal': 300000.0,
  'launched': datetime.datetime(2013, 9, 30, 18

## Question 6 

In [155]:
cur = collection_kickstarter.find({"name": {'$regex':'sport'}})
list(cur)

[{'_id': 369230992,
  'name': 'ULTIMMADMAN - action sports apparel   #JustBeTough',
  'category': 'Fashion',
  'main_category': 'Fashion',
  'currency': 'USD',
  'deadline': '2012-10-13',
  'goal': 5000.0,
  'launched': datetime.datetime(2012, 9, 10, 3, 48, 17),
  'pledged': 60.0,
  'state': 'failed',
  'backers': 1,
  'country': 'US',
  'usd pledged': 60.0,
  'usd_pledged_real': 60.0},
 {'_id': 1773369191,
  'name': '2Rare Taylor photo Magazine: Amazing Women and sports cars',
  'category': 'Photobooks',
  'main_category': 'Photography',
  'currency': 'USD',
  'deadline': '2017-10-23',
  'goal': 15000.0,
  'launched': datetime.datetime(2017, 9, 28, 1, 32, 49),
  'pledged': 65.0,
  'state': 'failed',
  'backers': 3,
  'country': 'US',
  'usd pledged': 0.0,
  'usd_pledged_real': 65.0},
 {'_id': 1453145888,
  'name': 'silentsportsusa celebrating the power of the human spirit!',
  'category': 'Journalism',
  'main_category': 'Journalism',
  'currency': 'USD',
  'deadline': '2013-01-06',
 